In [1]:
!pip3 install recommenders

     |████████████████████████████████| 335 kB 31.1 MB/s            
     |████████████████████████████████| 79 kB 1.7 MB/s             
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 310 kB 70.5 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 61 kB 677 kB/s             
     |████████████████████████████████| 11.8 MB 5.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.0 MB 64.5 MB/s            
     |████████████████████████████████| 2.0 MB 35.6 MB/s            
     |████████████████████████████████| 12.5 MB 40.1 MB/s            
     |████████████████████████████████| 377 kB 74.4 MB/s            
     |████████████████████████████████| 67 kB 7.1 MB/s             


     |███████████████████████████████▊| 6.6 MB 27.7 MB/s eta 0:00:01     |████████████████████████████████| 6.6 MB 27.7 MB/s            
     |████████████████████████████████| 880 kB 28.7 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.16-cp36-cp36m-linux_x86_64.whl size=762043 sha256=42825f7da56fa9037c524e07f40a39bdc396033a839cac61ef6c418cebd134aa
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6c/f0/48/ffe9095b572a6b4adde8fda07a20ea92b68ce33577d0e22adb
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31276 sha256=cc005ab225bcbfc4068353e82457258800b1f27a1ace2e28c136825f495b4ff4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fa/25/84/7bfb2bb7941fbd05a1cda2137057f22888c26189445c6032ec
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=ac2364cc9b55af688eac54154b46157e251701a7e695c9053dc37f488430e4fc
  Stored in directory: /home/ec2-us

In [1]:
# set the environment path to find Recommenders
import sys
import os

import boto3

import pyspark
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) 
[GCC 9.3.0]
Spark version: 2.4.0


## 0. Set up Spark Context

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap.

In [2]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = pyspark.sql.SparkSession \
        .builder \
        .config("spark.executor.instances", 10) \
        .config("spark.driver.memory", "16g") \
        .getOrCreate()
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

In [3]:
BUCKET_NAME = 'amazon-reviews-pds'
LOCAL_DIR = '../data/'
s3_client = boto3.client('s3')
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(BUCKET_NAME)

first = True

for object_summary in my_bucket.objects.filter(Prefix="parquet/product_category=Electronics/"):
    OBJECT_NAME = object_summary.key
    LOCAL_FILE = LOCAL_DIR + OBJECT_NAME.split("/")[-1]
    s3_client.download_file(BUCKET_NAME, OBJECT_NAME, LOCAL_FILE)
    data = spark.read.parquet(LOCAL_FILE)
    print(data.count())
    if first:
        first = False
    else:
        data = prev_data.union(data)
    prev_data = data

print(data.count())

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


312489
312064
311812
311393
312698
311962
312445
312888
311731
311456
3120938


In [4]:
# top k
TOP_K=10

DATA_PATH='../data/amazon_reviews_us_Electronics_v1_00.tsv'

COL_USER = "customer_id"
COL_ITEM = "product_parent"
COL_RATING = "star_rating"
COL_PREDICTION = "star_rating"
COL_TIMESTAMP = "review_date"
NUM_USERS = data.select(COL_USER).distinct().count()
NUM_ITEMS = data.select(COL_ITEM).distinct().count()

In [5]:
data = data.select(COL_USER, COL_ITEM, COL_RATING)

In [6]:
data.count()

3120938

In [7]:
data = data.withColumn(COL_USER,col(COL_USER).cast("integer"))
data = data.withColumn(COL_ITEM,col(COL_ITEM).cast("integer"))
data = data.withColumn(COL_RATING,col(COL_RATING).cast("integer"))

In [8]:
data.count()

3120938

In [9]:
import operator as op
from functools import reduce
import math
def ncr(n: int, r: int) -> int:
    """
    Calculates the number of different, unordered combination 
    of r objects from a set of n objects.
    nCr(n,r) = nPr(n,r) / r!
    
    Args:
        n (int): Number of objects in set
        r (int): Number of objects to combine
    Returns:
        int: Number of combinations
    """
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [10]:
data_sample = data.limit(5000)


In [11]:
n_chunks = 50
n=5000
nr_pairings = ncr(n, 2)
# minimum nr of pairings per chunk
per_chunk = int(math.ceil(nr_pairings/(n_chunks - 1)))

In [12]:
per_chunk

255052

In [13]:
split_indices = [] # Array containing the indices to split at
t = 0
for i in range(n + 1):
    # There are n - i pairings at index i
    t += n - i
    # If the current chunk has enough pairings 
    if t >= per_chunk:
        split_indices.append(i)
        t = 0

In [14]:
# We can check the number of pairings in each chunk running:

s_indices = [0] + split_indices + [n]
pairings_chunks = []
for i in range(len(s_indices)-1):
    start = s_indices[i]
    end = s_indices[i + 1]
    pairings_chunks.append(sum(range(n - end, n - start)))

In [18]:
%%timeit
ms = data_sample.groupby(COL_USER).mean(COL_RATING)
#data_sample = ms.join(data_sample, data_sample[COL_USER]==ms[COL_USER], 'inner')
#data_sample3 = data_sample2.withColumn('rating_norm_user', (data_sample2[COL_RATING] - data_sample2['avg(star_rating)']))

5.63 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
ms.show(10)

NameError: name 'ms' is not defined